In [ ]:
#PREPROCESSING STEPS REPEATED

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
value_counts = application_df["APPLICATION_TYPE"].value_counts()
cutoff = 156
application_types_to_replace = value_counts[value_counts <= cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
value_counts2 = application_df["CLASSIFICATION"].value_counts()
cutoff = 777
classifications_to_replace = value_counts2[value_counts2 <= cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(application_df, columns=["INCOME_AMT", "APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "SPECIAL_CONSIDERATIONS"])
numeric_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,True,False
1,1,108590,1,False,True,False,False,False,False,False,...,False,False,True,False,False,True,False,False,True,False
2,1,5000,0,True,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,True,False
3,1,6692,1,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,True,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,True,True,False


In [6]:
# Split our preprocessed data into our features and target arrays
X = numeric_df.drop(["IS_SUCCESSFUL"], axis=1).values
y = numeric_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
#Neural Network Model - Optimization Methods
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow Keras Tuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh'])

    # Allow Keras Tuner to decide the number of neurons in the first layer
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=1, max_value=30, step=5),
        activation=activation,
        input_shape=(X_train.shape[1],)  
    ))

    # Allow Keras Tuner to decide the number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=1, max_value=30, step=5),
            activation=activation
        ))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid")) 

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [9]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=5,
    hyperband_iterations=2)

Reloading Tuner from .\untitled_project\tuner0.json


In [ ]:
import numpy as np

# Convert boolean columns to integers
X_train = np.where(X_train == True, 1, X_train)
X_train = np.where(X_train == False, 0, X_train)

# Convert to float
X_train = X_train.astype(float)
X_test = X_test.astype(float)


X_train data types after conversion: float64
First few rows of X_train: [[1.e+00 5.e+03 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 1.e+00 0.e+00 0.e+00
  0.e+00 1.e+00 0.e+00]
 [1.e+00 5.e+03 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 0.e+00 0.e+00
  1.e+00 1.e+00 0.e+00]
 [1.e+00 5.e+03 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 1.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 0.e+00 0.e+00
  0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+00 0.e+00 0.e+00 0.e+00 0.e+00
  1.e+00 1.e+00 

In [13]:
# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=2, validation_split=0.2)

Trial 54 Complete [00h 00m 30s]
val_accuracy: 0.5446063876152039

Best val_accuracy So Far: 0.5446063876152039
Total elapsed time: 01h 52m 46s


In [18]:
#Get the Best Model
top_hyper = tuner.get_best_hyperparameters(1)[0]

In [21]:
# Build the model with the best hyperparameters
best_model = create_model(top_hyper)

c:\Anaconda\envs\dev\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
# Export our model to HDF5 file
best_model.save("AlphabetSoupCharity_Optimization.h5")